# Power management circuit

- Determine values of PMGMT circuit components using MFC energy data
- Estimate max number of samples that can be taken per cycle given the energy generated for each COD value

<br><img src="img/PIC18LF46K22_CODsensorC.png" alt="Drawing" style="width: 500px;"/>

In [1]:
import numpy as np
import pandas as pd

## Contraints of MFCs and Power MGMT components

Comparator: LTC1540 <br>https://www.analog.com/media/en/technical-documentation/data-sheets/1540fb.pdf
<br>Required to allow cold-start of the device for in-field operation. 

Regulator: TPS7A02 <br>https://www.ti.com/lit/ds/symlink/tps7a02.pdf?ts=1592828671056&ref_url=https%253A%252F%252Fwww.google.com%252F
Required to run the pic microcontroller at the lowest possible voltage thereby using the lowest possible power.

In [2]:
# Energy harvested from single batch feed MFC, Joules (J) for different COD values

Ea = np.array([21.137,  # COD = 900
                 18.017,  # COD = 500
                 8.315,   # COD = 300
                 2.415    # COD = 70
                ])

Ea *= 0.48 # apply energy harvesting eff.
Ea

array([10.14576,  8.64816,  3.9912 ,  1.1592 ])





Maximum and minimum rated supply voltages.

In [3]:
# max, min allowable supply voltage for regulator
Vmax = 6 
Vmin = 2.3 # lowest Vin for Vout = 1.8V 

Max energy stored in supply capacitor

In [4]:
E_cod900 = Ea[0]
E_cod900

10.14576

Capacitor energy $E = \frac{1}{2} CV^2$

Capacitance needed to store:

    `E_cod900` (max energy harvested) + 
    energy stored in capacitor at 2.3V @
    max voltage of camparator and regulator
    
$0.5CV_{max}^2 = 0.5CV_{min}^2 + E_{cod=900}$ 

$C = \frac{E_{max}}{\frac{1}{2} (V_{max}^2-V_{min}^2)}$

In [5]:
# C = E_cod900 / (0.5 * Vmax**2)
C = E_cod900 / (0.5 * (Vmax**2 - Vmin**2))

C = round(C, 2)
print(f'capacitor = {C} F')

capacitor = 0.66 F


The energy stored in capacitor $C$ at the lower cut-off voltage $V_{min}$ 

$E_{min} = \frac{1}{2} CV_{min}^2$

In [6]:
Emin = 0.5 * C * Vmin**2
Emin

1.7456999999999998

For each COD value, $E_a$ defines the energy budget accrued for use in the next COD estimation cycle

$E_{out} = nE_{sa} + E_{e} + E_{sl} + E_{r} + E_{c}$

where $n=$ number of samples

For each COD value, we can compute:
- the energy consumed by each of the main operations
    - sampling
    - COD estimation
    - sleep
    - voltage regulation 
    - comparator
    
- the maximum number of samples that can be taken given the amount of stored energy.
    
- the total energy stored in the capacitor, when charged by a single feed cycle of 1 MFC, assuming the starting voltage of the capacitor is $V_{min}$


 


## Energy used by each process

##### Micro-controller operations

In [7]:
processes = ['sample', 'estimate', 'sleep']


# execution time (s) [measured empirically]
week = 7*24*60*60
exec_time = [0.1, 0.3, week]


# current [measured empirically]
current = np.array([1.18e-3, 2.85e-3, 2e-8]) 


# voltage 
voltage = 1.8 # operting voltage of microcontroller 


# power
power = voltage * current
print(np.round(power,8))


# energy 
emc = power * exec_time 
print(np.round(emc,4))

# # total energy, microcontroller
# Emc = sum(emc)
# Emc

[2.124e-03 5.130e-03 4.000e-08]
[0.0002 0.0015 0.0218]


##### Power MGMT operations operations

The supply voltage to the comparator and voltage regulator is not constant. 
<br>It depends on the rate of discharge of the capacitor which depends on the resistance of the circuit, which is non-linear. 
<br>Deriving an equation for the power consumption precisely is therefore:
1. Beyond the scope of this study (we do not have a mathematical model for the dynamic behaviour of each component)
1. Of limited value since the amout of energy stored in the capacitor will vary with each cycle due to the stochasicity of the MFC behaviour. 

However, the following estimates can be made using information from the component data sheets about the relationship between voltage and current. 

###### 1.8V Regulator : TPS7A02
Quiescent current (IQ) is the current required to power the regulator's internal circuitry when the external load current is zero.
<br>Ground current (IGND) is the difference between the input and output currents, and necessarily includes the quiescent current. <br>A low ground current maximizes the LDO efficiency.

    IGND= IIN– IOUT

<table><tr><td> 
<img src='img/TPS7A02_ground.png' style="width: 400px;"> </td><td> 
<img src='img/TPS7A02_quiescent.png' style="width: 400px;"> </td><td> 
</table>
    
Assumptions:
- Ground current = :
    - 25nA @ 20nA ouput (microcontroller sleep = 20nA)
    - 10uA @ mA ouput (microcontroller estimate = 2.85mA)
    
- Voltage = `Vmax`
    




###### Comparator : LTC1540

<table><tr><td> 
<img src='img/IT_LTC1540.png' style="width: 300px;"> </td><td> 
<img src='img/IV_LTC1540.png' style="width: 300px;"> </td><td> 
</table>

Assumptions:
- Ground current = 0.3uA
- Voltage = `Vmax`

In [8]:
processes = ['comparator', 'regulator']

# execution time (s) 
exec_time = week 


# current 
current = np.array([25e-9, 3e-7]) 


# voltage 
voltage = Vmax 


# power
power = voltage * current
print(np.round(power,8))


# energy 
emgmt = power * exec_time 
print(np.round(emgmt,4))


# # total energy, power MGMT
# Emgmt = sum(emgmt)
# Emgmt

[1.5e-07 1.8e-06]
[0.0907 1.0886]


## Potential for energy autonomy 

The mimimun available energy needed to run the system can then be estimated as 
    
$E_{out} = nE_{sa} + E_{e} + E_{sl} + E_{r} + E_{c}$, where $n=1$

In [20]:
def Eout(n):
    return sum(emgmt) + sum(emc * np.array([n, 1, 1])) 


Eout(1)

1.2028842

In [21]:
n = 1

En1 = sum(emgmt) + sum(emc * np.array([n, 1, 1])) 

En1

1.2028842

We can then find the difference between `Ea` and `En1`, the difference between the average energy generated at each COD value and the minimum possible energy concumed by the system (i.e. the min mumber of samples, n=1).

In [22]:
Ea - En1

array([ 8.9428758,  7.4452758,  2.7883158, -0.0436842])

The vaule for COD=70 is negative suggesting there is not enough energy harvested when COD=70 for energy-autonomous behaviour as the energy used > energy generated. 

However, this is only an estimate so we won't worry too much about that for now. 

Using the energy consumption estimate for each process we can estimate the __maximum number of samples__ that can be taken in the next cycle, assuming only the energy harvested in the previous cycke is used.

In [23]:
# energy for one sampling event
Es = emc[0]
print(Es)

# energy for all other operations
Eops = sum(np.concatenate((emc[1:], emgmt)))
Eops

0.0002124


1.2026718

In [24]:
n_samples = (Ea - Eops) / Es
n_samples

array([42104.93502825, 35054.08757062, 13128.66384181,  -204.66949153])

And this can be used to find the time __minimum time increment between each sample__

In [25]:
t_inc = week / n_samples
print(f'minimum sampling increment = {t_inc} s')
print(f'({t_inc/60} min')

minimum sampling increment = [   14.36411194    17.2533374     46.06714036 -2955.0080742 ] s
([  0.23940187   0.28755562   0.76778567 -49.25013457] min


So we can see that, there is enough energy generated by each COD value (900, 500, 300) to run energy-autonomously and sample with a higher frequency than the arbitrary 2 mins chosen for testing, but possibly not enough by COD 70 to run energy-autonomously.




## Cold start

This functionality relies on the storage capacitor being charged above the cut-off voltage of `V_min`. 

The comparator is used to disconnect the supply to the voltage regulator (and the rest of the system) when the voltage of the storage capacitor drops to `Vmin` as it can no longer be regulated to 1.8V. 

A 'switch-on' voltage that is greater than this 'switch-off' voltage is introduced by setting a comparator hysteresis (next notebook). This enables the system to cold-start by allowing the storage capacitor to charge to a sufficient energy level to power the system before connecting it to the main circuit. This also allows the system to restart itself during operation of it loses power for example while operating in the field. 

The number of cycles required for the system to cold start will depend on the energy density (COD value) of the water samples fed to the MFC. 

<img src='img/accuracvstime.jpg' style="width: 400px;"> 

The question is what should this switch on voltage be? 

We can see from the plot of accuracy vs time, that there is little to be gained from choosing a sampling interval <32 mins and that accuracy begins to drop significantly >32. 

Therefore the sampling interval time was set as 32 mins (315 daya points over one week) and the comparator hysteresis was designed to switch-on the system only when there was enough energy stored in the capacitor to complete 315 samples in one week (estimated value). 

If the energy stored in the capacitor falls to the level where 315 data points cannot be collected, the system will shut down. Consequnelty the accuracy of prediction remains high. 

Again, the total energy in the storage capacitor will depend on the energy-density history of water samples - when the COD values are high there will be a surplus of energy that can help the system to continue to function even when the COD value is low. 

The total energy in the storage capacitor for 315 samples:

In [30]:
# Ecap needed for 32 samples

print(Eout(315))

E_cap = Eout(n) + 0.5*C*Vmin**2
E_cap

1.2695778


2.9485842

'Switch-on voltage of the system'

In [29]:
V_thl = Vmin
V_thh = np.sqrt(E_cap / (0.5*C))
print(V_thl, V_thh)

2.3 2.989164370917671


In [32]:
def Eout(n):
    return sum(emgmt) + sum(emc * np.array([n, 1, 1])) 


print(Eout(315))


def Eout(n):
    return sum(emgmt) + sum(emc * np.array([1, 1, n])) 


print(Eout(315))

1.2695778
8.0395434
